In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from selenium import webdriver
import time
import pandas as pd
import os

executable_path = {"executable_path":"/usr/local/bin/chromedriver"}
browser = Browser("chrome",**executable_path, headless = False)

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'html.parser')

In [3]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<!--[if lte IE 9]> <p class="browsehappy">You are using an <strong>outdated</strong> browser. Please <a href="http://browsehappy.com/">upgrade your browser</a> to improve your experience.</p> <![endif]-->
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":190,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var o=t[n]={exports:{}};e[n][0].call(o.exports,funct

In [10]:
latest_title=soup.find('div', class_='content_title').text.strip()
latest_title

latest_description= soup.find('div', class_="rollover_description_inner").text.strip()
latest_description


mars_latest_news = {
    'title': latest_title,
    'description': latest_description
    }


In [11]:
#print(mars_latest_news)
mars_latest_news['title']
mars_latest_news['description']

'If Mars has a global dust storm in 2018, observations could aid understanding of its effects.'

In [12]:
#results = soup.find_all('li', class_="slide")

#for result in results:
#    date = result.find('h3', class_='bottom_gradient').text
#    title = result.find('div', class_="list_date").text
#    description = result.find('div', class_="rollover_description_inner")
#    print(title)
#    print(date)
#    print(description)
    
#    Mars_News = {
        
#        'title': latest_title,
#        'description': latest_description
#    }


### Collecting featured image from Jet Propulsion Lab

In [13]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=featured#submit'

browser.visit(url)
time.sleep(3)  #allow time for page to load

In [14]:
# Retrieve page with the requests module
html = browser.html
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(html, 'html.parser')

#click full image button for feature picture on landing page(to second page)
browser.find_by_css("div.carousel_container div.carousel_items a.button").first.click()

time.sleep(3)  #allow time for page to load




In [15]:
#read in second page to beautifulsoup
html = browser.html
soup = bs(html,'html.parser')

#to third page
get_full_img = browser.find_by_xpath('//*[@id="fancybox-lock"]/div/div[2]/div/div[1]/a[2]').first.click()

time.sleep(3)  #allow time for page to load



In [16]:
#get large image
large_image_url = browser.find_by_xpath('//*[@id="page"]/section[1]/div/article/figure/a/img')["src"]
print(large_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14762_hires.jpg


### Mars weather

In [17]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'

browser.visit(url)
time.sleep(3)  #allow time for page to load


# Retrieve page 
html = browser.html
# Create BeautifulSoup object; parse with 'html'
soup = bs(html, 'html.parser')
#soup.prettify()



In [18]:
#browser.find_by_xpath('//*[@id="timeline"]/div')

In [19]:
#browser.find_by_xpath('//*[@id="stream-item-tweet-956593684564402176"]/div[1]/div[2]/div[3]/div/div/div/div/img).text

In [20]:
# skip tweet with pictures
#for result in results:
#    if result != browser.find_by_xpath('//*[@id="stream-item-tweet-956593684564402176"]/div[1]/div[2]/div[3]')
 #       print(soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text)
 #   else
        #hmmmm

In [21]:
#this is p tag looking for, but not for tweets with image
current_weather=soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
current_weather

'Sol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28'

In [22]:
#results.find('div', class_="AdaptiveMeidaOuterCountainer")

### Mars facts

In [23]:
url_table = 'https://space-facts.com/mars/'
table =  pd.read_html(url_table)

In [24]:
# create pandas DF
mars_table = table[0]
mars_table.columns = ["Parameter", "Values"]
mars_table.set_index(["Parameter"])



,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [27]:
#convert pandas DF to HTML file
mars_html_table = mars_table.to_html()


In [28]:
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Parameter</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronom